## Monash Handbook and Prequisites Documentation Project




### Determining prerequisites

I've written a script to scrape prerequisites from `https://mscv.apps.monash.edu` which validates a course map. Since it gives us prerequisites if we're missing them, we can simply submit up to 125 of those requests in a go. That being said, 
we still need a way to determine corequisites and prohibitions:
- As far as prohibitions are concerned, we can scrape from the handbook, then do a simple [A-Z]^3[0-9]^4 regex check on the prohibitions section. The API will return all the prohibited units.
- Corequisites are more tricky, however we should be able to make simplifications to our process. For instance, we can regex check the coreq section and then not include them on second pass for units that have a corequisite.

We now explore the data source. It takes 13.29 seconds on my PC to run this:

In [6]:
!python src/get_all_prereqs.py

13.293525218963623


Yields a file named `unit_reqs_clean.json`.

In [7]:
import json

In [8]:
with open("unit_reqs_clean.json", "r") as file:
    unit_reqs = json.load(file)

In [10]:
unit_reqs["FIT3047"]

[{'title': 'Not enough passed credit points',
  'description': 'You must pass 90 more credit points before you can enrol into this unit.'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 2 of these units: FIT1001 or FIT1005'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 1 of these units: FIT1040'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 1 of these units: FIT1008, FIT1035, FIT2034, FIT2071, FIT2076, FIT2081 or FIT2104'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 1 of these units: FIT1004 or FIT2094'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 2 of these units: FIT2001 or FIT2002'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 1 of these units: FIT1003, FIT1049 or FIT2003'}]